# Build the inputs of the UXsim model

### Prepare the environment

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.abspath("../.."), "functions"))

import data_reader
import spatial_assignment
import spatial_utils
import uxsimulator.preprocessing
from constants import P2V


### Load the data

In [ ]:
gdf_AreaVerde = data_reader.AV_shape(namefile="area_verde_manual_v1.geojson", datapath="../../data")
buffered_AreaVerde = spatial_utils.buffer_around(gdf=gdf_AreaVerde, buffer_size=3000)
areas_od = data_reader.OD_shapes(namefile_polygons="Shape_zone.SHP", namefile_centers="Shape_zone_centroid.SHP", datapath="../../data/OD")
aoi = data_reader.AOI_shapes(namefile="Shape_zone.SHP", datapath="../../data/OD", aoi_type="od", df_around=gdf_AreaVerde)
#aoi = data_reader.AOI_shapes(namefile="aree_statistiche.geojson", datapath="../../data", aoi_type="census", df_around=gdf_AreaVerde)

### Load or prepare the RN

In [ ]:
if os.path.exists(f"results/geo_edges_v5.geojson") and os.path.exists(f"results/geo_nodes_v5.geojson"):
    edges, nodes = data_reader.road_data(edges_namefile="geo_edges_v5.geojson", nodes_namefile="geo_nodes_v5.geojson", datapath="results")
else:
    import road_network
    edges, nodes = road_network.create_road_data(
        df_BAV=buffered_AreaVerde,
        relevant_highway=True, connected_network=True,
        edges_namefile="geo_edges_v5.geojson", nodes_namefile="geo_nodes_v5.geojson", datapath="../../data",
    )

### Transform the data

In [ ]:
# Assign out-zones to circular zones around AV
areas_od_av = spatial_assignment.OD_to_AV(df_od=areas_od, df_av=gdf_AreaVerde)
out_ids = areas_od_av.loc[(areas_od_av['mostly_within_area_verde']==False), 'id'].values
gdf_circles = uxsimulator.preprocessing.create_OD_circles(df_od=areas_od_av, df_bav=buffered_AreaVerde)
id_to_out_id_dict = spatial_assignment.OD_to_circles(df_od=areas_od_av, df_circles=gdf_circles)

In [ ]:
# Assign nodes to in/out AV
nodes_av = spatial_assignment.nodes_to_AV(df_nodes=nodes, df_av=gdf_AreaVerde)

In [ ]:
# Assign edges (i.e., roads) to the area of interest they mostly belong
edges_aoi = spatial_assignment.roads_to_AOI(df_edges=edges, df_aoi=aoi)

In [17]:
matrice_OD = data_reader.OD_flows(namefile="PROGETTO-OD.xlsx", datapath="../../data", df_shapes=areas_od_av[['id']])
matrice_OD['flow'] = matrice_OD['flow'] * P2V

matrice_OD = (
    matrice_OD
    .merge(areas_od_av[['id', 'mostly_within_area_verde']].rename(columns={'id': 'from', 'mostly_within_area_verde': 'from_area_verde'}), on='from', how='left')
    .merge(areas_od_av[['id', 'mostly_within_area_verde']].rename(columns={'id': 'to', 'mostly_within_area_verde': 'to_area_verde'}), on='to', how='left')
)

demand_in = uxsimulator.preprocessing.create_demand(
    od_matrix=matrice_OD[(matrice_OD['from_area_verde']==True) & (matrice_OD['to_area_verde']==True)], 
    circle_od_zones=gdf_circles, id_dict=id_to_out_id_dict, out_ids=out_ids)
demand_from_in = uxsimulator.preprocessing.create_demand(
    od_matrix=matrice_OD[(matrice_OD['from_area_verde']==True) & (matrice_OD['to_area_verde']==False)], 
    circle_od_zones=gdf_circles, id_dict=id_to_out_id_dict, out_ids=out_ids)
demand_to_in = uxsimulator.preprocessing.create_demand(
    od_matrix=matrice_OD[(matrice_OD['from_area_verde']==False) & (matrice_OD['to_area_verde']==True)], 
    circle_od_zones=gdf_circles, id_dict=id_to_out_id_dict, out_ids=out_ids)

### Save output

In [ ]:
suffix_output = 'v7'  # "census_v1"
dp = "results"

nodes_av.drop(['geometry'], axis=1).to_csv(f'{dp}/nodes_{suffix_output}.csv', header=False, index=False)
edges_aoi[['link_id', 'u', 'v', 'length', 'maxspeed_imputed', 'lanes_imputed', 'id_zone']]\
    .to_csv(f'{dp}/edges_{suffix_output}.csv',  header=False, index=False)
demand_in.to_csv(f'{dp}/flows_in_{suffix_output}.csv', index=False, header=False)
demand_from_in.to_csv(f'{dp}/flows_from_in_{suffix_output}.csv', index=False, header=False)
demand_to_in.to_csv(f'{dp}/flows_to_in_{suffix_output}.csv', index=False, header=False)

### Prepare the temporal weights for the demand

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def get_weights_from_hour_flows(inflow: list[float])->list[float]:
    inflow_mean = []
    for h in range(24):
        start = (np.array(range(25)) * 12)[h]
        end = (np.array(range(25)) * 12)[h+1]
        inflow_mean.append(inflow[start:end].mean())
    inflow_mean = inflow_mean / sum(inflow_mean)
    return inflow_mean

In [ ]:
# reference inflow
inflow = pd.read_parquet("../../data/inflows/inflow_v1.parquet")
inflow = np.array(inflow['inflow_weekday'].to_list())

In [ ]:
# modified inflow (after policy simulation)
inflow2 = np.array(pd.read_csv("../../data/veicoli in ingresso modificato.csv")['Area Verde'].to_list())
inflow3 = np.array(pd.read_csv("../../data/veicoli in ingresso modificato (1).csv")['Area Verde'].to_list())
inflow4 = np.array(pd.read_csv("../../data/veicoli in ingresso modificato (2).csv")['Area Verde'].to_list())

In [ ]:
# reference traffic
traffic =  pd.read_parquet("../../data/inflows/traffic_(1)_v1.parquet")
traffic = np.array(traffic['traffic'].to_list())

In [ ]:
get_weights_from_hour_flows(inflow)

In [ ]:
get_weights_from_hour_flows(traffic))